In [1]:
import os
# import zipfile2
from PIL import Image
from PIL.ExifTags import TAGS
from PIL.ExifTags import GPSTAGS
import requests
import folium
from folium.features import CustomIcon
from iptcinfo3 import IPTCInfo

# retirando o nome de cada arquivo do site

In [2]:
from urllib.request import urlopen
import urllib.request as request
from bs4 import BeautifulSoup
import io

import ssl
context = ssl._create_unverified_context()


br_site = "https://paisagemgrafica.joytill.com.br/fotos/fotos_insta/"
req = request.Request(br_site,
                        headers={'User-Agent': 'Mozilla'})
http = request.urlopen(req, context=context)
soup = BeautifulSoup(http, 'html.parser')

arquivos = []
for im in soup.findAll('a'):
    arquivos.append(im.get('href'))

arquivos.pop(0) 
arquivos.pop(0) 
arquivos.pop(0) 
arquivos.pop(0) 
arquivos.pop(0) 
print(arquivos)

['2016-02-08%2019.38.44-1.jpg', '2016-02-13%2008.52.21.jpg', '2016-02-14%2018.28.00-1.jpg', '2016-02-17%2013.34.52-1.jpg', '2016-02-17%2016.22.00-1.jpg', '2016-02-17%2017.48.20-1.jpg', '2016-02-26%2015.21.51-1.jpg', '2016-03-11%2011.55.16-1.jpg', '2016-03-17%2016.57.27-1.jpg', '2016-03-24%2011.12.51-1.jpg', '2016-03-30%2015.09.38-1.jpg', '2016-04-06%2015.54.03-1.jpg', '2016-04-19%2011.12.16-1.jpg', '2016-04-22%2009.16.56-1.jpg', '2016-04-27%2014.57.22-2.jpg', '2016-05-01%2009.50.43-2.jpg', '2016-05-01%2013.16.32-2.jpg', '2016-05-01%2014.04.36-2.jpg', '2016-05-07%2010.07.25-1.jpg', '2016-05-14%2010.12.52-1.jpg', '2016-05-22%2016.16.38-1.jpg', '2016-05-24%2016.54.00-2.jpg', '2016-05-26%2009.55.27-1.jpg', '2016-05-27%2013.36.47-1.jpg', '2016-05-28%2010.57.57-1.jpg', '2016-06-08%2019.01.50-1.jpg', '2016-06-18%2010.35.21-1.jpg', '2016-06-18%2010.57.54-1.jpg', '2016-06-22%2016.51.17-1.jpg', '2016-06-25%2014.24.01-2.jpg', '2016-06-26%2016.53.32-2.jpg', '2016-06-28%2018.34.16-2.jpg', '2016-06-

# obtento os metadados de cada imagem

In [12]:
arquivos_ = []
keywords = []
data = []
coor = []
for i in arquivos:
    newreq = request.Request(br_site+i,
                        headers={'User-Agent': 'Mozilla'})
    abriraurl = request.urlopen(newreq, context=context)
    try:
        arq_img = io.BytesIO(abriraurl.read())
        img = Image.open(arq_img)
    except UnidentifiedImageError:
        print('erro de imagem')
        pass
    #info = IPTCInfo(arq_img)
  #  print (info['keywords'])
    #for word in info['keywords']:
     #   keywords.append(word)
    try:
        exifdata = img.getexif()
        for tagid in exifdata:
            tagname = TAGS.get(tagid, tagid)
            value = exifdata.get(tagid)

        exif = {}
        for tag, value in img._getexif().items():
            if tag in TAGS:
                exif[TAGS[tag]] = value
    except AttributeError:
        print('sem dados')
        pass

#tentar obter de uma maneira única    
    try:
        datatempo=exif['DateTime']

        gps_info={}
        for k, v in exif['GPSInfo'].items():
            geo_tag=GPSTAGS.get(k)
            gps_info[geo_tag]=v

    #print(gps_info)

        lat=gps_info['GPSLatitude']
        long=gps_info['GPSLongitude']

    #print(lat)
    #print(long)

#verificar conversão
        lat=float(lat[0]+(lat[1]/60)+(lat[2]/(3600)))
        long=float(long[0]+(long[1]/60)+(long[2]/(3600)))

        if gps_info['GPSLatitudeRef']=='S':
            lat=-lat
        if gps_info['GPSLongitudeRef']=='W':
            long=-long
        coor.append([lat, long])
        data.append(datatempo)
        arquivos_.append(i)
    except KeyError:
        print('sem algum dado')

sem algum dado
sem algum dado
sem algum dado
sem algum dado
sem algum dado
sem dados
sem algum dado
sem algum dado
sem algum dado
sem dados
sem algum dado
sem dados
sem algum dado
sem dados
sem algum dado
sem dados
sem algum dado
sem dados
sem algum dado
sem algum dado
sem algum dado
sem dados
sem algum dado


# criando o mapa

In [13]:
cl = {'2016': 'red', '2017': 'blue', '2018': 'green', '2019': 'purple', '2020': 'orange', '2021': 'pink', '2022': 'black'}
m2=folium.Map(location=[lat,long],zoom_start=12) 
j = 0
tooltip = "Click me!"
for row in coor:
    d = data[j].split(':')
    d2 = d[2].split(' ')
    dat = d2[0] + '/' + d[1] + '/' + d[0]
    link = '<a href='+ br_site + arquivos_[j] +  '>Link para imagem</a>'
    popup = dat + '\n' + link
#folium.CircleMarker(location=[lat,long],fill=True, color='red',fill_color='red').add_to(m)
    marker = folium.Marker(location=row, popup=popup, tooltip=tooltip, icon=folium.Icon(color=cl[d[0]], icon='picture'))
    marker.add_to(m2)
    j += 1
m2.save('map_colors_.html')
m2